In [ ]:
import os
print("Current working directory:", os.getcwd())



Current working directory: d:\PROJECTS\cnns\creditCardFraud\models


In [1]:
import joblib
import xgboost as xgb
import numpy as np
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
from scipy.stats import randint,uniform
from sklearn.metrics import make_scorer, recall_score
from evaluation import evaluate_classification


In [3]:
data=joblib.load("D:\PROJECTS\cnns\creditCardFraud\data\without_smote_preprocessed_data.pkl")
X_train=data["X_train_scaled"]
X_test=data["X_test_scaled"]
y_train=data["y_train"]
y_test=data["y_test"]



<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\97798\AppData\Local\Temp\ipykernel_4436\2077961581.py:1: SyntaxWarning: invalid escape sequence '\P'
  data=joblib.load("D:\PROJECTS\cnns\creditCardFraud\data\without_smote_preprocessed_data.pkl")


In [ ]:


scorer = make_scorer(recall_score, average='binary')

param_grid= {
    'learning_rate': [0.01, 0.05],  # Lower rates for more boosting rounds
    'max_depth': [4, 5],            # Keep moderate depth
    'n_estimators': [200, 300],     # More trees to compensate for lower learning rate
    'min_child_weight': [5, 10],    # Maintain strong regularization
    'gamma': [0.1, 0.2],
    'subsample': [0.6, 0.8],        # Slightly more data per tree
    'colsample_bytree': [0.6, 0.8],
    'reg_alpha': [0.5, 1.0],        # Stronger L1 regularization
    'reg_lambda': [0.1, 0.5]
}

In [ ]:
xg_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='aucpr',  # Use PR-AUC as eval metric
    scale_pos_weight=100,
    early_stopping_rounds=10
)

# Use stratified k-fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configure search
random_search = RandomizedSearchCV(
    estimator=xg_model,
    param_distributions=param_grid,
    n_iter=50,
    scoring=scorer,
    cv=cv,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Fit RandomizedSearchCV to the training data
random_search.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)]
)

# Print the best hyperparameters found
print(f"\nBest hyperparameters: {random_search.best_params_}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits
[0]	validation_0-aucpr:0.61553
[1]	validation_0-aucpr:0.66498
[2]	validation_0-aucpr:0.69335
[3]	validation_0-aucpr:0.69660
[4]	validation_0-aucpr:0.71235
[5]	validation_0-aucpr:0.71189
[6]	validation_0-aucpr:0.72283
[7]	validation_0-aucpr:0.72262
[8]	validation_0-aucpr:0.72592
[9]	validation_0-aucpr:0.72486
[10]	validation_0-aucpr:0.72676
[11]	validation_0-aucpr:0.72953
[12]	validation_0-aucpr:0.72476
[13]	validation_0-aucpr:0.72368
[14]	validation_0-aucpr:0.72273
[15]	validation_0-aucpr:0.72000
[16]	validation_0-aucpr:0.72248
[17]	validation_0-aucpr:0.72323
[18]	validation_0-aucpr:0.72539
[19]	validation_0-aucpr:0.72494
[20]	validation_0-aucpr:0.72474

Best hyperparameters: {'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alpha': 0.5, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.6}


In [ ]:
# Get the best model
best_xg_model = random_search.best_estimator_


In [ ]:
print(best_xg_model)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=0.2, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


In [ ]:
# Make predictions on the test set
y_pred = best_xg_model.predict(X_test)
y_train_pred=best_xg_model.predict(X_train)

In [ ]:


from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix,auc

# Train metrics
print("Train metrics:")
print("Accuracy Score:", accuracy_score(y_train, y_train_pred))  # Corrected to y_train
print("Recall Score:", recall_score(y_train, y_train_pred, average="macro"))  # Corrected to y_train
print("Precision Score:", precision_score(y_train, y_train_pred, average="macro"))  # Corrected to y_train
print("F1 Score:", f1_score(y_train, y_train_pred, average="macro"))  # Corrected to y_train
print("Confusion matrix:", confusion_matrix(y_train, y_train_pred))  # Corrected to y_train

from sklearn.metrics import precision_recall_curve, auc

# Get predicted probabilities for the positive class (fraud)
y_proba = best_xg_model.predict_proba(X_train)[:, 1]

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_train, y_proba)

# Compute PR-AUC
pr_auc = auc(recall, precision)
print(f"PR-AUC: {pr_auc:.4f}")



# Test metrics
print("\nTest metrics:")
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("Recall Score:", recall_score(y_test, y_pred, average="macro"))
print("Precision Score:", precision_score(y_test, y_pred, average="macro"))
print("F1 Score:", f1_score(y_test, y_pred, average="macro"))
print("Confusion matrix:", confusion_matrix(y_test, y_pred))

from sklearn.metrics import precision_recall_curve, auc

# Get predicted probabilities for the positive class (fraud)
y_proba = best_xg_model.predict_proba(X_test)[:, 1]

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

# Compute PR-AUC
pr_auc = auc(recall, precision)
print(f"PR-AUC: {pr_auc:.4f}")


Train metrics:
Accuracy Score: 0.9994669133844392
Recall Score: 0.9046537856808728
Precision Score: 0.9308270627626765
F1 Score: 0.9173289957538422
Confusion matrix: [[226553     49]
 [    72    306]]
PR-AUC: 0.8294

Test metrics:
Accuracy Score: 0.9993303492757198
Recall Score: 0.863051983102449
Precision Score: 0.9256965074136168
F1 Score: 0.8918777818425705
Confusion matrix: [[56639    12]
 [   26    69]]
PR-AUC: 0.7536


Threshold Tuning

In [ ]:
y_proba = best_xg_model.predict_proba(X_test)[:, 1]
new_threshold = 0.30  # Lower = higher recall
y_pred = (y_proba >= new_threshold).astype(int)

In [ ]:
# Train metrics
print("Train metrics:")
print("Accuracy Score:", accuracy_score(y_train, y_train_pred))  # Corrected to y_train
print("Recall Score:", recall_score(y_train, y_train_pred, average="macro"))  # Corrected to y_train
print("Precision Score:", precision_score(y_train, y_train_pred, average="macro"))  # Corrected to y_train
print("F1 Score:", f1_score(y_train, y_train_pred, average="macro"))  # Corrected to y_train
print("Confusion matrix:", confusion_matrix(y_train, y_train_pred))  # Corrected to y_train

from sklearn.metrics import precision_recall_curve, auc

# Get predicted probabilities for the positive class (fraud)
y_proba = best_xg_model.predict_proba(X_train)[:, 1]

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_train, y_proba)

# Compute PR-AUC
pr_auc = auc(recall, precision)
print(f"PR-AUC: {pr_auc:.4f}")



# Test metrics
print("\nTest metrics:")
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("Recall Score:", recall_score(y_test, y_pred, average="macro"))
print("Precision Score:", precision_score(y_test, y_pred, average="macro"))
print("F1 Score:", f1_score(y_test, y_pred, average="macro"))
print("Confusion matrix:", confusion_matrix(y_test, y_pred))

from sklearn.metrics import precision_recall_curve, auc

# Get predicted probabilities for the positive class (fraud)
y_proba = best_xg_model.predict_proba(X_test)[:, 1]

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

# Compute PR-AUC
pr_auc = auc(recall, precision)
print(f"PR-AUC: {pr_auc:.4f}")

Train metrics:
Accuracy Score: 0.9994669133844392
Recall Score: 0.9046537856808728
Precision Score: 0.9308270627626765
F1 Score: 0.9173289957538422
Confusion matrix: [[226553     49]
 [    72    306]]
PR-AUC: 0.8294

Test metrics:
Accuracy Score: 0.9989602791386177
Recall Score: 0.8891382973682818
Precision Score: 0.8301717418612746
F1 Score: 0.8572275175325197
Confusion matrix: [[56613    38]
 [   21    74]]
PR-AUC: 0.7536


Threshold tuning also gave more worse result

In [ ]:
joblib.dump(best_xg_model, 'xgb_model.pkl')

['xgb_model.pkl']